<a href="https://colab.research.google.com/github/hantswilliams/sbu-minnesota-healthinsurance-allpayer/blob/main/scripts/analytics/Exploratory_MN_apcd_diagnoses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [ ]:
## Configure pandas to display non-exponential numbers
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Load the dataset (modify the filename if needed)
file_path = 'https://raw.githubusercontent.com/hantswilliams/sbu-minnesota-healthinsurance-allpayer/refs/heads/main/data/processed/diagnoses2021.csv'
df = pd.read_csv(file_path)

In [ ]:
### 1. Understanding Diagnosis Code Distribution ###
# Get the top 10 most common principal diagnosis codes
top_10_diagnoses = df['principal_diagnosis_code'].value_counts().nlargest(10)

# Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x=top_10_diagnoses.index, y=top_10_diagnoses.values, palette="viridis")
plt.xlabel("Principal Diagnosis Code")
plt.ylabel("Count")
plt.title("Top 10 Most Common Diagnosis Codes")
plt.xticks(rotation=45)
plt.show()

In [ ]:
### 2. Diagnosis vs. Total Paid Amount ###
# Group by diagnosis code and calculate the mean total paid amount
diagnosis_paid_mean = df.groupby('principal_diagnosis_code')['total_paid_amt_mean'].mean().nlargest(10)
print("Top 10 Diagnoses with Highest Mean Total Paid Amount:")
print(diagnosis_paid_mean)

# Boxplot for top 5 diagnoses
top_10_diagnoses = diagnosis_paid_mean.index
df_top_10 = df[df['principal_diagnosis_code'].isin(top_10_diagnoses)]

plt.figure(figsize=(12, 6))
sns.boxplot(x="principal_diagnosis_code", y="total_paid_amt_mean", data=df_top_10, palette="coolwarm")
plt.xlabel("Principal Diagnosis Code")
plt.ylabel("Total Paid Amount")
plt.title("Distribution of Total Paid Amount for Top 10 Diagnoses")
plt.xticks(rotation=45)
plt.show()

In [ ]:
### 3. Diagnoses by Age Group ###
# Create a contingency table
contingency_table = pd.crosstab(df['age_group_name'], df['principal_diagnosis_code'])

# Identify top 3 diagnoses for each age group
top_3_by_age = df.groupby('age_group_name')['principal_diagnosis_code'].value_counts().groupby(level=0).head(3)
print("Top 3 Diagnoses for Each Age Group:")
print(top_3_by_age)

# Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"\nChi-square Test Result: χ²={chi2:.2f}, p-value={p:.4f}")
if p < 0.05:
    print("Significant difference in diagnosis distribution across age groups.")
else:
    print("No significant difference in diagnosis distribution across age groups.")

In [ ]:
### 4. Insurer vs. Member Payment Differences ###
# Group by diagnosis code and calculate means
payment_means = df.groupby('principal_diagnosis_code')[['insurer_paid_amt_mean', 'member_paid_amt_mean']].mean()
print("Mean Payment Differences by Diagnosis:")
print(payment_means)

# Create a delta between insurer_paid_amt_mean and member_paid_amt_mean
payment_means['delta'] = payment_means['insurer_paid_amt_mean'] - payment_means['member_paid_amt_mean']
print("\nMean Payment Differences with Delta:")
print(payment_means)

# Top diagnoses with highest member out-of-pocket costs
top_member_paid = payment_means['member_paid_amt_mean'].nlargest(5)
print("\nTop 5 Diagnoses with Highest Member Out-of-Pocket Costs:")
print(top_member_paid)

# Top diagnoses with highest delta between insurer and member
top_delta = payment_means['delta'].nlargest(5)
print("\nTop 5 Diagnoses with Highest Delta Between Insurer and Member:")
print(top_delta)

# Scatter plot: Insurer Paid vs. Member Paid
plt.figure(figsize=(10, 6))
sns.scatterplot(x='insurer_paid_amt_mean', y='member_paid_amt_mean', data=payment_means, alpha=0.7)
plt.xlabel("Insurer Paid Mean")
plt.ylabel("Member Paid Mean")
plt.title("Insurer vs. Member Payment Differences")
plt.show()